In [5]:
print("hello World")

hello World


In [24]:
# Imports
import os
import docx
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
import numpy as np

# Function to load answers from Word documents
def load_answers_from_docs(doc_files):
    questions = []
    answers = []
    scores = []
    
    for doc_file in doc_files:
        doc = docx.Document(doc_file)
        for paragraph in doc.paragraphs:
            if paragraph.text.startswith("Q"):  # Assume question text starts with "Q"
                questions.append(paragraph.text) # store questions
            elif paragraph.text.startswith("A"):  # Assume answer text starts with "A"
                answers.append(paragraph.text[2:].strip())  # Skip "A:". store answers
            elif paragraph.text.startswith("Score"):  # Assume score starts with "Score:"
                scores.append([float(x) for x in paragraph.text.split(":")[1].strip().split(",")]) # store marking scheme
    return questions, answers, scores

# Load dataset
doc_files = ["Paper1.docx", "Paper2.docx", "Paper3.docx", "Paper4.docx", "Paper5.docx", "Paper6.docx", "Paper7.docx", "Paper8.docx", "Paper9.docx", "Paper10.docx"]
questions, answers, scores = load_answers_from_docs(doc_files)

# Preprocess text: Tokenization and padding
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(answers)
answers_seq = tokenizer.texts_to_sequences(answers)
answers_pad = tf.keras.preprocessing.sequence.pad_sequences(answers_seq, maxlen=500) # maxlen = max number of documents. Set to 500

# Convert scores to NumPy array for model compatibility
max_categories = 5 # Categories for marking scheme, e.g. Grammer, explainition, or example.
padded_scores = [] # used to fill out excess categories

for score in scores:
    # Pad with zeros or NaNs if the score list is shorter than max_categories
    if len(score) < max_categories:
        score += [0] * (max_categories - len(score))  # Padding with zeros
    padded_scores.append(score)

# Convert to numpy array to avoid breaking code
scores = np.array(padded_scores)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(answers_pad, scores, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=500),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5), # prevent overfitting of model
    LSTM(32),
    Dense(64, activation='relu'),
    Dense(len(score), activation='linear')  # Output matches the rubric categories. i.e. if 4 cate there will be 4 scored metrics. score comes fromfor loop.
])

# Compile the model
model.compile(optimizer='adam', # alan optimizer is used for training
              loss='mean_squared_error', # mean_squared_error is used for regression tasks
              metrics=['mae']) # Mean absolute error is used for trsining evaluation

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=10, # itations of training
          batch_size=32)

# Save the model as a keras file
model.save("essay_grading_model.keras")

# Final test and notification that process is finished
print("Model trained and saved successfully!")


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - loss: 5.9591 - mae: 1.9845 - val_loss: 5.6221 - val_mae: 1.9108
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 795ms/step - loss: 5.2399 - mae: 1.8412 - val_loss: 3.7291 - val_mae: 1.4927
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 798ms/step - loss: 3.2166 - mae: 1.4204 - val_loss: 2.0056 - val_mae: 1.1947
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 788ms/step - loss: 1.7031 - mae: 1.1099 - val_loss: 1.1892 - val_mae: 0.8994
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 789ms/step - loss: 0.9702 - mae: 0.7999 - val_loss: 0.7402 - val_mae: 0.6578
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 771ms/step - loss: 0.5674 - mae: 0.5856 - val_loss: 0.5684 - val_mae: 0.4936
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 782ms/step - loss: 0.4455 - mae: 0.4366 - val_loss: 0.4912 - val_mae: 0.3738
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 792ms/step - loss: 0.3463 - mae: 0.3224 - val_loss: 0.4755 - val_mae: 0.3317
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 822ms/step - loss: 0.3678 - mae: 0.

In [31]:
# imports
import tensorflow as tf
import docx
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os 
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Load the trained model
model = tf.keras.models.load_model("essay_grading_model.keras")
print("Model loaded successfully!")

# Load the tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
training_answers = answers
tokenizer.fit_on_texts(training_answers)

# Function to load and preprocess a new essay
def preprocess_new_essay(doc_file, tokenizer, max_len=500):
    # Step 1: Load the text from the .docx file
    doc = docx.Document(doc_file)
    essay_text = ""
    for paragraph in doc.paragraphs:
        essay_text += paragraph.text.strip() + " "
    
    # Step 2: Tokenize and pad the text
    sequence = tokenizer.texts_to_sequences([essay_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')
    
    return padded_sequence

# Entry of the new essay
new_essay_file = "testanswer.docx"  # In same folder
# process essay
padded_sequence = preprocess_new_essay(new_essay_file, tokenizer)

# Predict the rubric scores, i.e. grade answers
predicted_scores = model.predict(padded_sequence)

# Marked outputs. Used for Demo will replace in final version
rubric_categories = ["Grammar", "Coherence", "Content Depth", "Examples", "Language Clarity"]  # Adjust based on your rubric
max_scores = np.max(y_train, axis=0)

print("\nGrading Limits:", max_scores)
for i, score in enumerate(predicted_scores[0]):
    max_score = max_scores[i]  # Maximum score for the category
    # Error checking. Check for zero max_score
    if max_score == 0:
        percentage = 0  # Set the percentage to 0 if max_score is 0
        evaluation = "Nothing or Terribly answered"
        print(f"{rubric_categories[i]}: {score:.2f}: {evaluation}")
    else:
        percentage = (score / max_score) * 100  # Calculate percentage score
        if percentage == 100:
            evaluation = "Excellent"
        if percentage >= 50:
            evaluation = "Good"
        else:
            evaluation = "Poor, needs improvement"
        print(f"{rubric_categories[i]}: {score:.2f}: {evaluation}")


Model loaded successfully!
Preprocessed essay shape: (1, 500)
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Scores: [[ 0.9702759   2.199916    3.1293108   3.2702675  -0.00480052]]

Grading Results: [4. 5. 4. 4. 0.]
Grammar: 0.97: Poor, needs improvement
Coherence: 2.20: Poor, needs improvement
Content Depth: 3.13: Good
Example: 3.27: Good
Language Clarity: -0.00: Nothing
